This notebook is adapted from [a lesson from the 2017 KIPAC/StatisticalMethods course](https://github.com/KIPAC/StatisticalMethods/blob/2017winter/chunks/generative_models.ipynb), (c) 2017 Adam Mantz and Phil Marshall. The GPLv2 license applies.

# Evaluating Models

Goals:

* Be able to design and carry out goodness of fit and model comparison tests

* Understand and be prepared to use the Bayesian Evidence

Three related but distinct questions come under the heading of **model evaluation**.
1. Does a model describe (fit) the data well?
2. Does a model make accurate predictions about new data?
3. Is a model probable in light of the data?

Often (2) and (3) are directly related to **model comparison** or **selection**.

**A familiar example:** imagine we have a data set like this
<img src="../graphics/modelcheck-data.png" width=60%>

Specifically,
* we have precisely known $x$ values
* we have precisely known, Gaussian errors on $y$
* we're fitting a linear model, $\bar{y}(x)=b+mx$

In this case, the likelihood is $\propto e^{-\chi^2/2}$.

So is the posterior, given uniform priors on $a$ and $b$.

Assuming this model is correct, the distribution over data sets of $\hat{\chi}^2$ must follow a $\chi^2_\nu$ distribution, where
* $\hat{\chi}^2$ is the best-fit $\chi^2$ over parameters for a given data set
* the number of degrees of freedom $\nu=N_\mathrm{data}-N_\mathrm{params}$

Hence, the classical $\chi^2$ test looks at whether $\hat{\chi}^2$ is consistent with this distribution. If not, it's unlikely that our data came from the assumed model.

In this case, the value of $\hat{\chi}^2\approx104$ doesn't look good in light of the expectation.
<img src="../graphics/modelcheck-chisq.png" width=50%>
The probability $P(\chi^2\geq\hat{\chi}^2|\nu)$ ($\sim10^{-10}$ in this case) is called the **$p$-value** or **significance**.

More generally, our likelihood won't have these nice, analytic properties. We can still construct analogous tests by simulating many mock data sets realized from the posterior distribution. Then
1. Compare the real data directly with the distribution of simulated data, or
2. Compare a summary statistic of the data with the distribution of summary statistics

Either way, there is some freedom to choose what aspect of the data we care about the model fitting well.

This is called **posterior predictive** checking.

Visual comparison of draws from the posterior with the data. (NB: no simulated data here...)
<table>
    <tr>
        <td><img src="../graphics/modelcheck-linear-posterior.png " width=90%></td>
        <td></td>
        <td><img src="../graphics/modelcheck-linear.png" width=90%></td>
    </tr>
</table>

Example test statistic: Pearson Correlation $r_{12}$

A single number we might chose to summarize the data set is

$T(d) = r_{12} = \frac{\sum_i (x_i - \bar{x})(y_i - \bar{y})}{\left[ \sum_i (x_i - \bar{x})^2 \sum_i (y_i - \bar{y})^2 \right]^{1/2}}$

For many posterior samples, we draw a replica dataset from the sampling distribution given yhe sample parameter vector, and compute $T(d^{\rm rep})$, building up a histogram of $T$.

${\rm Pr}(T(d^{\rm rep})>T(d)\,|\,d) = 99.43\%$

<img src="../graphics/modelcheck-linear-TS.png" width=50%>

Test statistics $T(d,\theta)$ that are functions of both the data and the parameters are called **discrepancy measures**.

The log-likelihood is a common example.

Discrepancy measure: $T = \hat{\chi}^2$

${\rm Pr}(T(d^{\rm rep},\theta)>T(d,\theta)\,|\,d) \approx 0.0$

<img src="../graphics/modelcheck-linear-discrepancy.png" width=50%>

Any way we look at it, it's unlikely that we'd conclude the linear model fits these data well. How do we choose an alternative?

One way to compare the fitness of models is to look at question (2) in model evaluation: **How accurately do they predict new data?**

* Implicitly, this assumes we want a fit that works well with any *potential* data set, rather than just reproducing the one we have.
* In general, this means an "Occam's Razor"-like penalty for complexity should be involved.

In our example, we might add a quadratic term to the model: $y = b + m x + q x^2$. How do we quantify the improvement?

<table><tr>
<td><img src="../graphics/modelcheck-quadratic.png" width=80%></td>
<td><img src="../graphics/modelcheck-quadratic-discrepancy.png" width=80%></td>
</tr></table>

The gold standard for testing predictive accuracy is to get more data.

Short of that, the best option is **cross-validation**: fitting a model on many random subsets of the data and seeing how well it describes the complementary "out of sample" subsets.
* This method is ubiquitous in machine learning, where accurate out-of-sample prediction is usually the goal.

Short of exhaustive cross-validation, a number of **information criteria** exist that (asymptotically) relate to predictive accuracy.

These have the advantage of being relatively quick to calculate from the results of a fit - either an MLE or a set of posterior samples - and include a penalty for models with greater freedom.

Some information criteria
* Akaike information criterion (AIC)
* Deviance information criterion (DIC)
* Watanabe-Akaike information criterion (WAIC)

The DIC has the advantage of being compatible with Bayesian analysis (unlike AIC), and not requiring the data to be cleanly separable into conditionally independent subsets (unlike WAIC).

$\mathrm{DIC} = \langle D(\theta) \rangle + p_D; \quad p_D = \langle D(\theta) \rangle - D(\langle\theta\rangle)$

where $D(\theta)=-2\log P(\mathrm{data}|\theta)$ and averages $\langle\rangle$ are over the posterior.

$p_D$ is an _effective number of free parameters_, i.e. the number of parameters primarily constrained by the data rather than by their priors.

The DIC thus doesn't necessarily count nuisance parameters used to marginalize out systematics as added "complexity".

Note that **lower** IC is preferable (larger likelihood and/or less model complexity).

A somewhat motivated scale for interpretting differences in IC exists (named for Jeffreys):

| $e^{\mathrm{IC}_1-\mathrm{IC}_2}$  | Strength of evidence for model 2 |
|--------|:------------------------:|
|  < 1   | Negative |
|  1-3   | Barely worth mentioning |
|  3-10  | Substantial |
| 10-30  | Strong |
| 30-100 | Very strong |
|   >100 | Decisive |

How about the third question - **Is a model probable in light of the data?**

This is an application of Bayes theorem.

Inference on parameters $\theta$ given model $H$:

$P(\theta|D,H)=\frac{P(D|\theta,H)P(\theta|H)}{P(D|H)}$

Inference on models $H$: $P(H|D,\Omega)=\frac{P(D|H,\Omega)P(H|\Omega)}{P(D|\Omega)}$

Here $\Omega$ is some space of all allowed models. As we normally do for parameter inference, we'll work with a simplified version:

$P(H|D)\propto P(D|H)P(H)$

Here $P(H)$ is a prior on the model, and

$P(D|H)=\int P(D|\theta,H) \, P(\theta|H) d\theta$ is the **evidence** from Bayesian parameter inference (also known as the **fully marginalized likelihood**).

Ideally, we would compare models by looking at

$\frac{P(H_1|D)}{P(H_2|D)}=\frac{P(D|H_1)\,P(H_1)}{P(D|H_2)\,P(H_2)}$

In fact, a very general method called **Metropolis-Hastings-Green** sampling can be used to produce Markov chains that traverse models with completely different parameter spaces.

General difficulties in working with the evidence:
* assigning meaningful priors to models
* assigning meaningful priors to parameters

**Exercise: "non-informative" parameter priors and the evidence**

Say we have a model with 1 parameter, $\theta$, and a likelihood that works out to be a unit Gaussian in $\theta$.

For each of the priors on $\theta$ below, (a) sketch the likelihood and prior as a function of theta, (b) calculate the evidence (well enough to comment about the relative differences, anyway).
1. $P(\theta)$ uniform on $[-1,+1]$
2. $P(\theta)$ uniform on $[-3,+3]$
3. $P(\theta)$ uniform on $[-100,+100]$
4. $P(\theta)$ uniform on $[-\infty,+\infty]$
5. $P(\theta)$ uniform on $[+9,+12]$

Models that are less prescriptive (in terms of their priors) are penalized in the evidence. This is a feature, although it means we need to put real thought into those priors.

Assigning priors to models is perhaps even harder. Often people punt on this by working with the **evidence ratio** aka **Bayes factor**, which can be plugged into the Jeffreys scale.

$R = \frac{P(D|H_1)}{P(D|H_2)}$


When two models are nested or can be connected by a new, continuous parameter, another sensible option is to simply do parameter inference in that more general model.

In the example above, we can compute the evidence for the linear and quadratic models, and form the odds ratio $R$.

```
log Evidence for Straight Line Model: -157.2
log Evidence for Quadratic Model: -120.7
Evidence ratio in favour of the Quadratic Model:
  7e15 to 1
```

The 26 unit difference in log evidence between the two models translates to a _huge_ odds ratio in favour of the quadratic model.

**Calculating the evidence**

Estimates directly calculated from Markov chains produced for parameter inference are generally not reliable.

Good methods include nested sampling (e.g. [MultiNest](https://github.com/JohannesBuchner/PyMultiNest)) and parallel tempering (e.g. [emcee](http://dan.iel.fm/emcee/current/)).

We'll look at a simple (and not generally recommended) method in the problem.